In [1]:
import os
import numpy as np
from omegaconf import DictConfig, OmegaConf
import hydra
from hydra import initialize, compose
from hydra.utils import instantiate
from starfish.data import StarfishDataModule
from starfish.model import FasterRCNNLightning

import torch
from pprint import pprint
# Set the environment variable HYDRA_FULL_ERROR=1 to see the full stack trace
os.environ['HYDRA_FULL_ERROR'] = '1'

os.environ['PROJECT_ROOT'] = '/Users/moustholmes/starfishDetection'

# Change the current working directory to the project root
os.chdir('/Users/moustholmes/starfishDetection')

initialize(version_base=None, config_path='configs', job_name='notebook')

cfg_train = compose(config_name='main_config',) #overrides=['experiment=effect_gaussian_nll'])

print(OmegaConf.to_yaml(cfg_train))

# from pytorch_lightning.loggers import WandbLogger
# logger = instantiate(cfg_train.logger)

model:
  _target_: starfish.model.FasterRCNNLightning
  num_classes: 2
  log_ap: true
  learning_rate: 0.001
  momentum: 0.9
  weight_decay: 0.0005
  optimizer:
    _target_: torch.optim.Adam
    _partial_: true
    lr: 0.001
    weight_decay: 0.01
  scheduler:
    _target_: torch.optim.lr_scheduler.StepLR
    _partial_: true
    step_size: 3
    gamma: 0.1
  compile: false
data:
  _target_: starfish.data.StarfishDataModule
  data_path: data/raw
  batch_size: 4
  train_val_test_split:
  - 0.8
  - 0.1
  - 0.1
  subset: 0.1
  num_workers: 1
trainer:
  _target_: pytorch_lightning.Trainer
  min_epochs: 1
  max_epochs: 2
  accelerator: cpu
  devices: 1
logger:
  _target_: pytorch_lightning.loggers.WandbLogger
  offline: false
  id: null
  anonymous: null
  project: CorruptMNIST
  log_model: false
  prefix: ''
  entity: MoustHolmes
  group: ''
  tags:
  - test
  job_type: ''



In [2]:
# from pytorch_lightning.loggers import WandbLogger
logger = instantiate(cfg_train.logger)

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [2]:
data = instantiate(cfg_train.data)
print(data)
data.setup()
train_dataloader = data.train_dataloader()
batch = next(iter(train_dataloader))
x, y = batch
assert len(x) == data.batch_size
assert len(y) == data.batch_size

Loading 492 images.


In [3]:
model = instantiate(cfg_train.model)
print(model)

FasterRCNNLightning(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
from pytorch_lightning.loggers import WandbLogger
logger = instantiate(cfg_train.logger)

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [5]:
trainer = instantiate(cfg_train.trainer, logger=logger)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(


In [6]:
trainer.fit(model, data)


  | Name  | Type       | Params
-------------------------------------
0 | model | FasterRCNN | 41.3 M
-------------------------------------
41.1 M    Trainable params
222 K     Non-trainable params
41.3 M    Total params
165.197   Total estimated model params size (MB)


/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (2) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  rank_zero_warn(


Epoch 0: 100%|██████████| 2/2 [00:47<00:00, 23.90s/it, v_num=pbnx, train_loss_classifier=0.0497, train_loss_box_reg=0.011, train_loss_objectness=2.570, train_loss_rpn_box_reg=0.0146, train_total_loss=2.650, val_AP=0.000]

/Users/moustholmes/miniconda3/envs/starfish/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:212: UserWarning: You called `self.log('val_AP', ...)` in your `validation_step` but the value needs to be floating point. Converting it to torch.float32.
  warning_cache.warn(


Epoch 1: 100%|██████████| 2/2 [00:58<00:00, 29.36s/it, v_num=pbnx, train_loss_classifier=0.0377, train_loss_box_reg=0.0107, train_loss_objectness=1.950, train_loss_rpn_box_reg=0.055, train_total_loss=2.050, val_AP=0.000]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 2/2 [00:58<00:00, 29.49s/it, v_num=pbnx, train_loss_classifier=0.0377, train_loss_box_reg=0.0107, train_loss_objectness=1.950, train_loss_rpn_box_reg=0.055, train_total_loss=2.050, val_AP=0.000]


In [5]:
import torchvision
import torch

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='DEFAULT')
# For training
images, boxes = torch.rand(4, 3, 600, 1200), torch.rand(4, 11, 4)
boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
labels = torch.randint(1, 91, (4, 11))
images = list(image for image in images)
targets = []
for i in range(len(images)):
    d = {}
    d['boxes'] = boxes[i]
    d['labels'] = labels[i]
    targets.append(d)
output = model(images, targets)
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)
# optionally, i

is mps available? True


In [5]:
import torch
print(f'is mps available? {torch.backends.mps.is_available()}')
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model.to(device)

is mps available? True


FasterRCNNLightning(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [4]:
from torch.optim import Adam
# data = instantiate(cfg_train.data)

data.setup()
train_dataloader = data.train_dataloader()

# optimizer = Adam(model.parameters(), lr=0.001)  # Initialize the optimizer

In [5]:


num_epochs = 1  # Number of epochs to train for

for epoch in range(num_epochs):
    # model.train()  # Set the model to training mode
    # running_loss = 0.0

    for batch in train_dataloader:
        images, targets = batch
        print(targets)
        print([im.shape for im in images])

[{'boxes': tensor([[153.5000, 474.6667, 176.0000, 515.5555],
        [262.5000, 551.1111, 285.0000, 602.6667]]), 'labels': tensor([1, 1])}, {'boxes': tensor([[202.5000, 196.4444, 219.5000, 224.0000]]), 'labels': tensor([1])}, {'boxes': tensor([[587.0000, 538.6666, 607.5000, 571.5555]]), 'labels': tensor([1])}, {'boxes': tensor([[207.5000, 409.7778, 233.0000, 451.5555]]), 'labels': tensor([1])}]
[torch.Size([3, 640, 640]), torch.Size([3, 640, 640]), torch.Size([3, 640, 640]), torch.Size([3, 640, 640])]
[{'boxes': tensor([[218.5000, 273.7778, 233.5000, 300.4445]]), 'labels': tensor([1])}, {'boxes': tensor([[370.5000, 373.3333, 388.5000, 406.2222],
        [383.5000, 397.3333, 401.0000, 426.6667],
        [394.5000, 411.5555, 414.5000, 445.3333]]), 'labels': tensor([1, 1, 1])}, {'boxes': tensor([[417.0000, 338.6667, 455.5000, 400.8889]]), 'labels': tensor([1])}, {'boxes': tensor([[182.5000, 233.7778, 199.0000, 260.4445]]), 'labels': tensor([1])}]
[torch.Size([3, 640, 640]), torch.Size([3,

In [17]:
from torchmetrics.detection import MeanAveragePrecision
from torch import tensor

metric = MeanAveragePrecision(iou_type="bbox")


print(f'is mps available? {torch.backends.mps.is_available()}')
device = torch.device("cpu") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

# preds = [
#   dict(
#     boxes=tensor([[258.0, 41.0, 606.0, 285.0]]),
#     scores=tensor([0.536]),
#     labels=tensor([0]),
#   )
# ]
# target = [
#   dict(
#     boxes=tensor([[214.0, 41.0, 562.0, 285.0]]),
#     labels=tensor([0]),
#   )
# ]
metric = MeanAveragePrecision(iou_type="bbox")
# metric.update(preds, target)
metric.tm_to_coco("tm_map_input")

num_epochs = 1  # Number of epochs to train for

for epoch in range(num_epochs):
    # model.train()  # Set the model to training mode
    model.eval()

    for batch in train_dataloader:
        images, targets = batch
        # move images and targets to the device
        # images = list(image.to(device) for image in images)
        # targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        # print(targets)
        # print(target)
        # print([im.shape for im in images])
        # print()
        # print(preds)
        output = model(images)  # Forward pass
        # print(output)

        metric.update(output, targets)

        break
print(metric.compute())

is mps available? True
{'map': tensor(0.), 'map_50': tensor(0.), 'map_75': tensor(0.), 'map_small': tensor(0.), 'map_medium': tensor(0.), 'map_large': tensor(-1.), 'mar_1': tensor(0.), 'mar_10': tensor(0.), 'mar_100': tensor(0.), 'mar_small': tensor(0.), 'mar_medium': tensor(0.), 'mar_large': tensor(-1.), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor(1, dtype=torch.int32)}


In [20]:
from torchmetrics.detection import MeanAveragePrecision
from torch import tensor

from torchvision import models, datasets, ops
# from torchvision.models.detection import efficientdet_lite0

model = models.detection.fasterrcnn_mobilenet_v3_large_fpn(weights="DEFAULT")


# model = efficientdet_lite0(pretrained=True)


data = instantiate(cfg_train.data)
data.setup()
train_dataloader = data.train_dataloader()

# model = instantiate(cfg_train.model)
# model = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
device = torch.device("mps") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

model.train() 

num_epochs = 1  # Number of epochs to train for


# time it
import time
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")



Loading 492 images.
Time taken: 168.8138370513916


In [22]:
model = models.detection.fasterrcnn_resnet50_fpn(weights="DEFAULT")
device = torch.device("mps") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

model.train() 

num_epochs = 1  # Number of epochs to train for


# time it
import time
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")


Time taken: 934.0693581104279


In [25]:
model = models.detection.ssdlite320_mobilenet_v3_large(weights="DEFAULT")
device = torch.device("mps") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

model.train() 

num_epochs = 1  # Number of epochs to train for


# time it
import time
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")

Time taken: 10.989444255828857


In [ ]:
model = models.detection.ssdlite320_mobilenet_v3_large_fpn(weights="DEFAULT")
device = torch.device("mps") # torch.device("mps") if torch.backends.mps.is_available() else 
model.to(device)

model.train() 

num_epochs = 1  # Number of epochs to train for


# time it
import time
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")

In [26]:
start = time.time()


for epoch in range(num_epochs):
    
    i=0

    for batch in train_dataloader:
        i+=1
        images, targets = batch
        # move images and targets to the device
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # output = model(images,targets)  # Forward pass

        if i%10==0:
            break


print(f"Time taken: {time.time()-start}")

Time taken: 9.8573579788208
